<a href="https://colab.research.google.com/github/SeoMinJong/deep_learning/blob/main/deep_learning_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신경망 함수 정리

In [ ]:
%run /content/drive/MyDrive/DeepLearning/MataUtils.ipynb

In [ ]:
print(RND_MEAN, RND_STD, LEARNING_RATE)

0 1 0.07


In [ ]:
    def main_execute(epoch_count = 10, mb_size = 10, report = 2, train_ratio = 0.8):
        load_dataset()
        weight_initial, bias_initial = init_param() # 가중치를 초기화 시켜 반환해준다.
        losses_mean_row, accs_mean_row, final_acc = train_and_test(epoch_count, 
                                                                   mb_size, 
                                                                   report, 
                                                                   train_ratio)

        return weight_initial, bias_initial, losses_mean_row, accs_mean_row, final_acc

In [ ]:
def load_dataset():
    with open('/content/drive/MyDrive/database/abalone.csv') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)

        rows = []
        for row in csv_reader:
            rows.append(row)

    global input_cnt, output_cnt, data
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows),input_cnt + output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'M': data[n, 0] = 1
        if row[0] == 'F': data[n, 1] = 1
        if row[0] == 'I': data[n, 2] = 1
        data[n, 3:] = row[1:]

In [ ]:
def init_param():
    global weight, bias
        
    weight_initial = []
    bias_initial = []  # 학습할 때의 과정 과정을 확인할 수 있다.

    weight = np.random.normal(RND_MEAN, RND_STD, size= [input_cnt, output_cnt])
    bias = np.zeros([output_cnt]) # 바이어스의 초기값이 생각보다 굉장히 영향이 크기 때문에 초기값을 0로 잡는 것
    print("initial weight Value:\n{}".format(weight))
    print("initial Bias Value:\n{}".format(bias))

    weight_initial.append(weight)
    bias_initial.append(bias)

    return weight_initial, bias_initial

In [ ]:
def train_and_test(epoch_count, mb_size, report, train_ratio):
    mini_batch_step_count = arrange_data(mb_size = mb_size, train_ratio = train_ratio)

    test_X, test_y = get_test_data()

    losses_mean_row = []
    accs_mean_row = []

    for epoch in range(epoch_count):
        losses = []
        train_accs = []
        for nth in range(mini_batch_step_count):
            train_X, train_y = get_train_data(mb_size, nth)
            
            loss, train_acc = run_train(train_X,train_y)

            losses.append(loss)
            train_accs.append(train_acc)

        if report > 0 and (epoch +1) % report == 0:
            test_acc = run_test(test_X, test_y)
            print("Epoch {} : Train - Loss = {:.3f}, Accuracy = {:.3f} / Test - Accuracy = {:.3f}".\
                  format(epoch+1,np.mean(losses),np.mean(train_accs),test_acc))
        
        losses_mean = np.mean(losses)
        train_accs_mean = np.mean(train_accs * 100)

        losses_mean_row.append(losses_mean)
        accs_mean_row.append(train_accs_mean)

    final_acc = run_test(test_X,test_y)
    print("="*30,"Final TEST","="*30)
    print("Final ACccuracy = {:.3f}".format(final_acc))

    return losses_mean_row, accs_mean_row, final_acc

In [ ]:
def arrange_data(mb_size, train_ratio):
    global shuffle_map, test_begin_index
    
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)

    mini_batch_step_count = int(data.shape[0] * train_ratio) // mb_size
    test_begin_index = mini_batch_step_count * mb_size

    return mini_batch_step_count

In [ ]:
def get_test_data():
    test_data = data[shuffle_map[test_begin_index:]]

    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

In [ ]:
# nth가 0과 같아 지면 shuffle을 하게 된다.

def get_train_data(mb_size ,nth):
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_index])
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth +1)]]
    # nth = 미니 배치의 순서

    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [ ]:
def forward_neuralnet(x):
    y_hat = np.matmul(x, weight)+bias

    return y_hat, x

In [ ]:
def forward_postproc(y_hat,y):
    diff = y_hat - y
    square = np.square(diff)
    loss = np.mean(square)

    return loss, diff

In [ ]:
def backprop_neuralnet(G_output, x):
    global weight, bias 

    x_transpose = x.transpose()
    G_w = np.matmul(x_transpose, G_output)

    G_b = np.sum(G_output, axis = 0)

    weight -= LEARNING_RATE * G_w
    bias   -= LEARNING_RATE * G_b
    

In [ ]:
def backprop_postproc(diff):
    M_N = diff.shape

    g_mse_square  = np.ones(M_N) / np.prod(M_N) # mse를 square로 미분 / np.prod(변수) = 변수 안에 있는 원소들을 곱해주는 함수
    g_square_diff = 2 * diff # diff를 square로 미분
    g_diff_output = 1

    G_diff = g_mse_square * g_square_diff
    G_output = g_diff_output * G_diff

    return G_output

In [ ]:
def run_train(x, y):
    y_hat, aux_nn_x   = forward_neuralnet(x)
    loss, aux_pp_diff = forward_postproc(y_hat, y)

    accuracy = eval_accuracy(y_hat, y)

    G_output = backprop_postproc(aux_pp_diff)
    backprop_neuralnet(G_output, aux_nn_x)

    return loss, accuracy

In [ ]:
def run_test(X, y):
    y_hat, _ = forward_neuralnet(X)
    accuracy = eval_accuracy(y_hat, y)

    return accuracy

In [ ]:
def eval_accuracy(y_hat, y):
    mdiff = np.mean(np.abs((y_hat - y) / y))
    
    return 1 - mdiff